# **Notebook 5b - Maps of DiverCity**

---

## **Overview**
This notebook generates **city-level maps of DiverCity**, showing the spatial distribution of potential route diversification across urban areas.  
These maps visually highlight how network structure and mobility attractors influence local variations in DiverCity,  
revealing areas with high or low potential for route diversification.

The resulting maps can be used for visual analysis, cross-city comparison, and inclusion in reports or online visualizations (e.g., the [DiverCity interactive platform](https://divercitymaps.github.io)).

---

The notebook produces:
- **DiverCity Heatmaps** - Continuous spatial representations of node-level DiverCity values.  
- **Overlay Maps** - Visualizations including attractors and boundaries for context.  
- **Exported Figures** - Maps for selected cities.

These maps correspond to the visualizations used in *Figure 2a–b* of the DiverCity paper and on the [DiverCity interactive platform](https://divercitymaps.github.io).

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from route_plotting import process_single_city, compute_gpd_divercity_nodes, create_geodesic_circle
from my_utils import create_folder_if_not_exists

## **1. Parameters**

This section defines the parameters used to generate the DiverCity map for a selected city.  
Each parameter determines how the data will be loaded and visualized.

In [ ]:
# ------------------------------------------------------------
# Map generation parameters
# ------------------------------------------------------------

city = "milan"          # City to visualize
radius_km = 10         # Visualization radius (must match the computed DiverCity data)
k = 5                  # Number of near-shortest routes used for DiverCity calculation
exp_id = "exp_osm"     # Experiment identifier (used for loading results)

## **2. Generate the DiverCity Map**

This section creates a **continuous spatial map of DiverCity** for the selected city.  

In [ ]:
# ------------------------------------------------------------
# Load and prepare DiverCity data for visualization
# ------------------------------------------------------------

df_cities = pd.read_csv("../data/city_info.csv")

# Process the selected city and load its DiverCity results
_, res_city = process_single_city(city, radius_km * 1000, exp_id, k)

# Retrieve city coordinates
c_lat, c_lng = df_cities[df_cities["city"] == city][["lat", "lng"]].values[0]

# Compute GeoDataFrame of nodes with DiverCity values
nodes_city = compute_gpd_divercity_nodes(
    res_city["divercity"],
    res_city["road_networks"]["dict_sampling"],
    k=k
)

# Prepare grid for interpolation
points = np.array([(geom.x, geom.y) for geom in nodes_city.geometry])
values = nodes_city["divercity"].values

x_min, x_max = points[:, 0].min(), points[:, 0].max()
y_min, y_max = points[:, 1].min(), points[:, 1].max()
grid_x, grid_y = np.mgrid[x_min:x_max:100j, y_min:y_max:100j]

# Interpolate DiverCity values (linear method)
grid_z = griddata(points, values, (grid_x, grid_y), method="linear", fill_value=np.nan)

# ------------------------------------------------------------
# Create geodesic circles and clip attractors for visualization
# ------------------------------------------------------------

circle_viz = create_geodesic_circle((c_lat, c_lng), radius_km=radius_km)
gpd_circle_viz = gpd.GeoDataFrame(geometry=[circle_viz], crs="EPSG:4326")

# Clip attractor edges within visualization boundary
attractor_city = res_city["road_networks"]["attractor_edges"]
clipped_attractors = gpd.overlay(attractor_city, gpd_circle_viz)

# ------------------------------------------------------------
# Plot DiverCity map
# ------------------------------------------------------------

fig, ax = plt.subplots(figsize=(4, 4), dpi=200)

# Contour plot of interpolated DiverCity values
levels = 12
contour = ax.contourf(grid_x, grid_y, grid_z, levels=levels, cmap="Blues")

# Overlay mobility attractors
clipped_attractors.plot(color="orange", linewidth=1, ax=ax, zorder=10)
clipped_attractors.plot(color="#FFD580", linewidth=0.5, ax=ax, zorder=10)

# Remove axis for a clean map aesthetic
ax.axis("off")

# Adjust plot boundaries
xmin, ymin, xmax, ymax = gpd_circle_viz.total_bounds
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)

# Add colorbar
colorbar = fig.colorbar(contour, ax=ax, fraction=0.02, pad=0.05, orientation="horizontal")
colorbar.ax.tick_params(labelsize=4)
colorbar.set_label("DiverCity", fontsize=8)

# ------------------------------------------------------------
# Save figure
# ------------------------------------------------------------
create_folder_if_not_exists("./Figures/")
save_path = f"./Figures/divercity_map_{city}.pdf"
plt.savefig(save_path, bbox_inches="tight")